# colab vm에 google drive mount

In [ ]:
### colab에 내 구글 drive mount하기

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth

auth.authenticate_user()

from oauth2client.client import GoogleCredentials

creds = GoogleCredentials.get_application_default()

import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL

vcode = getpass.getpass()

!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
### mount할 drive 생성, 만든 계정 연결

!mkdir -p /opt/bin/daehan/drive
!google-drive-ocamlfuse /opt/bin/daehan/drive

In [ ]:
### drive mount

from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/opt/bin/daehan/drive')

In [ ]:
### data upzip

path_to_zip_file = '/opt/bin/daehan/data/train/[train] wav.zip'
directory_to_extract_to = '/opt/bin/daehan/data/train'

import zipfile
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

# 전처리에 필요한 함수

In [11]:
# 전처리에 필요한 module
import os
import json
import numpy as np
import pandas as pd
import shutil
from tqdm import tqdm

# 경로 설정

BASE_DIR = '/opt/bin/daehan/'
TRAIN_DIR = '/opt/bin/daehan/data/train/'
LABEL_DIR = '/opt/bin/daehan/data/train_json/'
TEST_DIR = '/opt/bin/daehan/data/test/'

# 경로 내의 확장자별 파일 분리
def file_filter(path):
    input = sorted(os.listdir(path))
    jjson = []
    wav = []
    flac = []

    for n in input:
        if '.flac' in n:
            flac.append(n)
        elif '.wav' in n:
            wav.append(n)
        else:
            jjson.append(n)
    return sorted(jjson), sorted(flac), sorted(wav)

# json 파일의 메타데이터에서 파일명, 대본 라벨, 대본, 전사 추출
def labeling(label):

    '''
    Filename: 음성데이터 파일 이름      >> list
    SentenceID: 대본 라벨               >> list
    SpeakerID: 화자 라벨                >> list
    Utter_txt: 한글만 있는 대본         >> list
    Gender: 화자 성별                    >> list
    label >> sorted(os.listdir(path))결과
    '''

    Filename = []
    SpeakerID = []
    Utter_txt = []
    SentenceID = []
    Gender = []


    for n in label:
        if n[-4:] != 'json':
            continue
    with open(LABEL_DIR + n) as f:
        label_info = json.load(f)

    Filename.append(label_info['fileName'])
    SentenceID.append(label_info['file_info']['sentenceID'])
    SpeakerID.append(label_info['file_info']['speakerID'])
    Utter_txt.append(label_info['transcription']['ReadingLabelText'])
    Gender.append(label_info['basic_info']['gender'])

    return Filename, SentenceID, SpeakerID, Utter_txt, Gender

# 파일명 전사내용 trans.txt파일로 저장
def make_file(x,SentenceID, SpeakerID, Utter_txt):
    # label txt maker
    text_file_path = '{}.trans.txt'.format(x+'_004')
    with open(text_file_path, 'w', newline = '') as f:
        for i in range(len(SentenceID)):
            tmp = SpeakerID[i]+'_004_'+SentenceID[i]+' '+Utter_txt[i]+'\n'
            f.write(tmp)
     

    # 결과에 '\n'와 중복값 제거 
    new_text_content=''
    with open(text_file_path,'r') as g:

        lines = g.readlines()
        for i, l in enumerate(list(set(lines))):
            if l == '\n':
                new_text_content += ''
            elif l[:20] in new_text_content:
                new_text_content += ''
            else:
                new_text_content += l

    # 다시 저장
    with open(text_file_path,'w') as f:
        f.write(new_text_content)



# dataset.flac -> Speechdata/train_data_01/004/, /test_data_01/004

In [ ]:

# {'CN11RC001_CN0003_20210730', 'CN11RC001_CN0006_20210827', 'CN11RC001_CN0001_20210827'} : 누락 파일

# DIR
BASE_DIR = '/opt/bin/daehan/'
TRAIN_DIR = '/opt/bin/daehan/data/train/'
LABEL_DIR = '/opt/bin/daehan/data/train_json/'
TEST_DIR = '/opt/bin/daehan/data/test/'
SAVE_DIR = './model_saved/'

# train,val 분할
json_list = sorted(os.listdir(LABEL_DIR))
json_list.remove('CN11RC001_CN0003_20210730_SW071.json')
json_list.remove('CN11RC001_CN0001_20210827_SW076.json')
json_list.remove('CN11RC001_CN0006_20210827_SW053.json')
tr = os.listdir('/opt/bin/daehan/zeroth/s5/speechDATA/train_data_01/004/')
te = os.listdir('/opt/bin/daehan/zeroth/s5/speechDATA/test_data_01/004')

label_val = []

for n in json_list:
  for k in te:
    if k in n:
      label_val.append(n)
label_train = list(set(json_list).difference(set(label_val)))

_, SentenceID1, SpeakerID1, Utter_txt1, _ = labeling(label_train)
_, SentenceID2, SpeakerID2, Utter_txt2, _ = labeling(label_val)

# train 파일 flac변환 후 dataset에 저장

os.chdir('/opt/bin/daehan/zeroth/s5/speechDATA/train_data_01/')

train_zip = sorted(list(zip(SentenceID1, SpeakerID1, Utter_txt1)), key = lambda x: (x[1],x[0],x[2]))
val_zip = sorted(list(zip(SentenceID2, SpeakerID2, Utter_txt2)), key = lambda x: (x[1],x[0],x[2]))

for i in range(len(label_train)):
  if os.path.exists('./004/'+SpeakerID1[i]+'/') == False:
    os.mkdir('./004/'+SpeakerID1[i]+'/')

  shutil.copy(LABEL_DIR + label_train[i], './004/'+SpeakerID1[i]+'/'+SpeakerID1[i]+'_004_'+SentenceID1[i]+'.json')
  shutil.copy(TRAIN_DIR+flac_train[i],'./004/'+SpeakerID1[i]+'/'+SpeakerID1[i]+'_004_'+SentenceID1[i]+'.flac')

# data/train_data_01 폴더에 text, utt2spk, wav.scp 업데이트 >> 필요 x

  # with open('/opt/bin/daehan/zeroth/s5/data/train_data_01/text', 'a') as g:
  #   g.write('{} {}\n'.format(train_zip[i][1]+'_004_'+train_zip[i][0], train_zip[i][2]))
  # with open('/opt/bin/daehan/zeroth/s5/data/train_data_01/utt2spk', 'a') as h:
  #   h.write('{} {}\n'.format(train_zip[i][1]+'_004_'+train_zip[i][0], train_zip[i][1]+'_004'))
  # with open('/opt/bin/daehan/zeroth/s5/data/train_data_01/wav.scp', 'a') as j:
  #   j.write('{0} flac -c -d -s ./speechDATA/train_data_01/004/{1}/{0}.flac |\n'.format(train_zip[i][1]+'_004_'+train_zip[i][0], train_zip[i][1]))

# data/test_data_01 ~~

  # for i in range(len(label_val)):
  # with open('/opt/bin/daehan/zeroth/s5/data/test_data_01/text', 'a') as g:
  #     g.write('{} {}\n'.format(val_zip[i][1]+'_004_'+val_zip[i][0], val_zip[i][2]))
  # with open('/opt/bin/daehan/zeroth/s5/data/test_data_01/utt2spk', 'a') as h:
  #   h.write('{} {}\n'.format(val_zip[i][1]+'_004_'+val_zip[i][0], val_zip[i][1]+'_004'))
  # with open('/opt/bin/daehan/zeroth/s5/data/test_data_01/wav.scp', 'a') as j:
  #   j.write('{0} flac -c -d -s ./speechDATA/test_data_01/004/{1}/{0}.flac |\n'.format(val_zip[i][1]+'_004_'+val_zip[i][0], val_zip[i][1]))

# Audioinfo 생성

In [ ]:
# flac 변화 실패한 data를 제외
json_list = sorted(os.listdir(LABEL_DIR))
json_list.remove('CN11RC001_CN0003_20210730_SW071.json')
json_list.remove('CN11RC001_CN0001_20210827_SW076.json')
json_list.remove('CN11RC001_CN0006_20210827_SW053.json')
tr = os.listdir('/opt/bin/daehan/zeroth/s5/speechDATA/train_data_01/004/')
te = os.listdir('/opt/bin/daehan/zeroth/s5/speechDATA/test_data_01/004')

# train set에서 speakerID가 곂치지 않게 약 10% val set을  분할

label_val = []

for n in json_list:
  for k in te:
    if k in n:
      label_val.append(n)
label_train = list(set(json_list).difference(set(label_val)))

# metadata에서 필요한 data 추출
_, _, SpeakerID1, _, Gender1 = labeling(label_train)
_, _, SpeakerID2, _, Gender2 = labeling(label_val)

train_lst = list(set(zip(SpeakerID1,Gender1)))
val_lst = list(set(zip(SpeakerID2, Gender2)))
# train_set
with open('/opt/bin/daehan/zeroth/s5/speechDATA/AUDIO_INFO', 'a') as f:
    for i in range(len(train_lst)):
      f.write('{}|{}|{}|004|train_data_01\n'.format(train_lst[i][0],train_lst[i][0],train_lst[i][1].lower()))
# val_set
with open('/opt/bin/daehan/zeroth/s5/speechDATA/AUDIO_INFO', 'a') as f:
    for i in range(len(val_lst)):
      f.write('{}|{}|{}|004|test_data_01\n'.format(val_lst[i][0], val_lst[i][0], val_lst[i][1].lower()))


# speakerID별 trans.txt 생성

In [6]:
cd ../

C:\Users\lmsoo\Desktop\cslee공모전


In [ ]:
# DIR 설정
# BASE_DIR = '/opt/bin/daehan/'
# TRAIN_DIR = '/opt/bin/daehan/data/train/'
LABEL_DIR = './json/'
# TEST_DIR = '/opt/bin/daehan/data/test/'

# _ , flac = file_filter(TRAIN_DIR) 
json_list = sorted(os.listdir(LABEL_DIR))
# json_list.remove('CN11RC001_CN0003_20210730_SW071.json')
# json_list.remove('CN11RC001_CN0001_20210827_SW076.json')
# json_list.remove('CN11RC001_CN0006_20210827_SW053.json')
# tr = os.listdir('/opt/bin/daehan/zeroth/s5/speechDATA/train_data_01/004/')
# te = os.listdir('/opt/bin/daehan/zeroth/s5/speechDATA/test_data_01/004')

# train set에서 speakerID가 곂치지 않게 약 10% val set을  분할

# label_val = []

# for n in json_list:
#     for k in te:
#         if k in n:
#             label_val.append(n)
# label_train = list(set(json_list).difference(set(label_val)))
label_train = json_list[:10000]

# trainset과 val. set에 해당하는 json파일에서 필요한 요소 추출, 정렬
_, SentenceID1, SpeakerID1, Utter_txt1, _ = labeling(label_train)
# _, SentenceID2, SpeakerID2, Utter_txt2, _ = labeling(label_val)

train_zip = sorted(list(zip(SentenceID1, SpeakerID1, Utter_txt1)), key = lambda x: (x[1],x[0],x[2]))
# val_zip = sorted(list(zip(SentenceID2, SpeakerID2, Utter_txt2)), key = lambda x: (x[1],x[0],x[2]))

In [14]:
json_list

['CN11RC001_CN0001_20210827_SW076.json',
 'CN11RC001_CN0003_20210730_SW071.json',
 'CN11RC001_CN0006_20210827_SW053.json',
 'CN11RC001_CN0024_20210730_SW059.json',
 'CN11RC001_CN0025_20210730_SW005.json',
 'CN11RC001_CN0029_20210730_SW019.json',
 'CN11RC001_CN0031_20210726_SW025.json',
 'CN11RC001_CN0035_20210726_SW047.json',
 'CN11RC001_CN0036_20210831_SW070.json',
 'CN11RC001_CN0038_20210824_SW014.json',
 'CN11RC001_CN0041_20210730_SW071.json',
 'CN11RC001_CN0042_20210727_SW045.json',
 'CN11RC001_CN0043_20210730_SW052.json',
 'CN11RC001_CN0044_20210731_SW012.json',
 'CN11RC001_CN0045_20210727_SW075.json',
 'CN11RC001_CN0046_20210730_SW059.json',
 'CN11RC001_CN0050_20210727_SW033.json',
 'CN11RC001_CN0051_20210727_SW025.json',
 'CN11RC001_CN0053_20210730_SW045.json',
 'CN11RC001_CN0058_20210726_SW071.json',
 'CN11RC001_CN0065_20210730_SW059.json',
 'CN11RC001_CN0067_20210727_SW047.json',
 'CN11RC001_CN0072_20210801_SW075.json',
 'CN11RC001_CN0075_20210730_SW103.json',
 'CN11RC001_CN00

In [ ]:
# train trans.txt생성
for x in tqdm(sorted(os.listdir('/opt/bin/daehan/zeroth/s5/speechDATA/train_data_01/004/'))):
    temp = []
    LABEL_DIR = '/opt/bin/daehan/zeroth/s5/speechDATA/train_data_01/004/'+ x +'/'
    os.chdir(LABEL_DIR)
    for i in train_zip:
        if x == i[1]:
            temp.append(i)
        else:
            continue
        SentenceID, SpeakerID, Utter_txt = zip(*temp)
#         print(len(SentenceID),len(SpeakerID),len(Utter_txt))
        make_file(x, SentenceID, SpeakerID, Utter_txt)


  4%|▎         | 5/138 [00:00<00:02, 45.21it/s]

167 167 167
265 265 265
861 861 861
743 743 743
817 817 817
542 542 542
257 257 257
382 382 382
216 216 216
562 562 562
294 294 294
374 374 374
360 360 360


 14%|█▍        | 20/138 [00:00<00:01, 60.92it/s]

973 973 973
210 210 210
618 618 618
463 463 463
8 8 8
920 920 920
208 208 208
902 902 902
370 370 370
821 821 821
651 651 651
872 872 872


 24%|██▍       | 33/138 [00:00<00:01, 53.15it/s]

...

112 112 112
68 68 68
173 173 173
357 357 357
161 161 161
11 11 11
332 332 332
93 93 93
123 123 123
7 7 7
344 344 344
353 353 353
355 355 355
122 122 122
110 110 110
365 365 365
149 149 149
71 71 71
396 396 396
175 175 175
325 325 325
326 326 326


 90%|████████▉ | 124/138 [00:01<00:00, 106.06it/s]

1 1 1
9 9 9
1 1 1
168 168 168
286 286 286
376 376 376
256 256 256
183 183 183
17 17 17
158 158 158
148 148 148
330 330 330
186 186 186
150 150 150
51 51 51
318 318 318
3 3 3
167 167 167
1 1 1
272 272 272
245 245 245
39 39 39
74 74 74
170 170 170


100%|██████████| 138/138 [00:01<00:00, 88.04it/s] 

55 55 55
194 194 194
76 76 76
173 173 173
181 181 181
94 94 94
25 25 25
285 285 285
173 173 173
151 151 151
184 184 184


In [ ]:
# test trans.txt 생성
for x in sorted(os.listdir('/opt/bin/daehan/zeroth/s5/speechDATA/test_data_01/004/')):
  temp = []
  LABEL_DIR = '/opt/bin/daehan/zeroth/s5/speechDATA/test_data_01/004/'+ x +'/'
  os.chdir(LABEL_DIR)
  for i in val_zip:
    if x == i[1]:
      temp.append(i)
    else:
      continue
  SentenceID, SpeakerID, Utter_txt = zip(*temp)
  # print(len(SentenceID),len(SpeakerID),len(Utter_txt))
  make_file(x,SentenceID, SpeakerID, Utter_txt)

167 167 167
265 265 265
861 861 861
743 743 743
817 817 817
542 542 542
257 257 257
382 382 382
216 216 216
562 562 562
294 294 294
374 374 374
360 360 360
973 973 973
210 210 210
618 618 618
463 463 463
8 8 8
920 920 920
208 208 208
902 902 902
370 370 370
821 821 821
651 651 651
872 872 872
577 577 577
2 2 2
921 921 921
761 761 761
918 918 918
844 844 844
308 308 308
999 999 999
368 368 368
237 237 237
269 269 269
944 944 944
602 602 602
670 670 670
64 64 64
359 359 359
182 182 182
135 135 135
174 174 174
5 5 5
360 360 360
360 360 360
320 320 320
107 107 107
179 179 179
87 87 87
320 320 320
373 373 373
243 243 243
375 375 375
389 389 389
117 117 117
191 191 191
83 83 83
261 261 261
4 4 4
350 350 350
202 202 202
343 343 343
296 296 296
374 374 374
5 5 5
163 163 163
21 21 21
77 77 77
73 73 73
354 354 354
192 192 192
314 314 314
175 175 175
166 166 166
248 248 248
309 309 309
278 278 278
327 327 327
141 141 141
112 112 112
68 68 68
173 173 173
357 357 357
161 161 161
11 11 11
332 332 33

# Debug용 메모장


In [ ]:
# 성별 확인
a = []
with open('/opt/bin/daehan/zeroth/s5/speechDATA/AUDIO_INFO', 'r') as f:
  lines = f.readlines()
  for line in lines:
    line = line.strip('\n').split('|')
    a.append(line) 
gender = [i[2] for i in a]
foo = [i for i in sex if i != 'f' and i !='m']

In [ ]:
train_dir = set(os.listdir('/opt/bin/daehan/zeroth/s5/speechDATA/train_data_01/004/'))
test_dir = set(os.listdir('/opt/bin/daehan/zeroth/s5/speechDATA/test_data_01/004/'))
same = list(train_dir.intersection(test_dir))

for n in same:
  shutil.rmtree('/opt/bin/daehan/zeroth/s5/speechDATA/train_data_01/004/'+n+'/')

In [ ]:
CN = [n for n in sorted(label_train) if 'CN0001' in n]